In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

import statsmodels.formula.api as smf

from matplotlib import pyplot as plt

# Import data

This data comes from this paper: https://www.nature.com/articles/s41467-019-13483-w

It's a big excel file with expression and growth rate data in different sheets and this code will use a few of these sheets.

In [3]:
df = pd.ExcelFile(r'../Data/raw_data/41467_2019_13483_MOESM4_ESM.xlsx')

## Basic metadata clean up and data subsetting

After some manual inspection, looking at distributions, etc. we decided to exclude samples where:

1. The growth rate data was unknown


2. The growth rate data was reported as zero. This one might seem strange but it's is a little unclear in general if that's possible/true to have zero growth rate. It is possible that these were stationary phase cultures but equally likely from my stand-point that these are errors in the table.


3. Really poor alignment (perhaps indicating some overall contamination)

In [4]:
meta_df = df.parse('Metadata') ###This grabs the sheet that contains information about the samples
print(meta_df.shape)
meta_df = meta_df[meta_df['Growth Rate (1/hr)'].isnull() == False]
print(meta_df.shape)
meta_df = meta_df[meta_df['Growth Rate (1/hr)'] > 0.0]
print(meta_df.shape)
meta_df = meta_df[meta_df['Alignment'] > 80]
print(meta_df.shape)
meta_df.head()

(278, 26)
(195, 26)
(179, 26)
(173, 26)


,Sample ID,Study,Project ID,Condition ID,Replicate #,Strain Description,Strain,Base Media,Carbon Source (g/L),Nitrogen Source (g/L),...,Culture Type,Growth Rate (1/hr),Evolved Sample,Isolate Type,Sequencing Machine,Additional Details,Biological Replicates,Alignment,DOI,GEO
4,fur__wt_fe__1,Fur,fur,wt_fe,1,Escherichia coli K-12 MG1655,MG1655,M9,glucose(2),NH4Cl(1),...,Batch,1.060606,No,NaN,MiSeq,NaN,2,93.35,doi.org/10.1038/ncomms5910,GSE54900
5,fur__wt_fe__2,Fur,fur,wt_fe,2,Escherichia coli K-12 MG1655,MG1655,M9,glucose(2),NH4Cl(1),...,Batch,1.060606,No,NaN,MiSeq,NaN,2,92.38,doi.org/10.1038/ncomms5910,GSE54900
8,fur__delfur_fe2__1,Fur,fur,delfur_fe2,1,Escherichia coli K-12 MG1655 del_fur,MG1655,M9,glucose(2),NH4Cl(1),...,Batch,0.619469,No,NaN,MiSeq,NaN,2,92.80,doi.org/10.1038/ncomms5910,GSE54900
9,fur__delfur_fe2__2,Fur,fur,delfur_fe2,2,Escherichia coli K-12 MG1655 del_fur,MG1655,M9,glucose(2),NH4Cl(1),...,Batch,0.619469,No,NaN,MiSeq,NaN,2,93.24,doi.org/10.1038/ncomms5910,GSE54900
55,omics__bw_ac__1,Omics,omics,bw_ac,1,Escherichia coli BW25113,BW25113,M9,acetate(3.5),NH4Cl(1),...,Batch,0.203000,No,NaN,MiSeq,NaN,2,97.80,doi.org/10.1038/ncomms13091,GSE59759


## Read in the expression data

In [5]:
exp_df = df.parse('Expression Data', index_col='log-TPM')
print(exp_df.shape)
exp_df = exp_df[meta_df['Sample ID']] ###Only grab the columns corresponding to the samples identified above
print(exp_df.shape)
assert list(meta_df['Sample ID']) == list(exp_df.columns) ###Check our work
exp_df.head()

(3923, 278)
(3923, 173)


,fur__wt_fe__1,fur__wt_fe__2,fur__delfur_fe2__1,fur__delfur_fe2__2,omics__bw_ac__1,omics__bw_ac__2,omics__bw_fum__1,omics__bw_fum__2,omics__bw_glc__1,omics__bw_glc__2,...,efeU__menFentC_ale29__1,efeU__menFentC_ale29__2,efeU__menFentC_ale30__1,efeU__menFentC_ale30__2,efeU__menFentCubiC_ale36__1,efeU__menFentCubiC_ale36__2,efeU__menFentCubiC_ale37__1,efeU__menFentCubiC_ale37__2,efeU__menFentCubiC_ale38__1,efeU__menFentCubiC_ale38__2
log-TPM,,,,,,,,,,,,,,,,,,,,,
b0002,11.064810,10.779071,11.229767,11.214065,9.257348,9.182322,9.709213,9.672126,10.208587,10.218351,...,10.271327,10.276565,11.148538,11.170578,11.676604,11.726097,11.881529,11.923237,11.496410,11.552762
b0003,10.776984,10.597810,10.897938,10.861157,8.983408,8.943151,9.436004,9.394573,9.609637,9.677931,...,10.160291,10.116861,10.314322,10.392251,10.916426,10.909277,11.023924,11.038426,10.624301,10.764195
b0004,10.394971,10.113950,10.185151,10.164655,8.761690,8.779920,9.532673,9.534370,9.883558,9.870356,...,10.475069,10.434352,10.679541,10.723953,11.143310,11.112721,11.184795,11.241845,10.953206,11.001006
b0005,6.716069,6.410864,6.527653,6.136168,4.474204,4.720490,5.782102,5.326669,5.846675,5.972022,...,5.979079,5.705586,6.306120,6.291340,5.058537,4.835550,5.448097,5.757951,5.873964,5.808618
b0006,6.761813,6.816532,6.862147,6.817480,6.536457,6.439917,6.408731,6.276017,6.910200,6.843384,...,8.371287,8.322390,8.137515,8.071837,7.354131,7.365536,7.328101,7.314761,8.057750,8.105213


**Manual inspection found some weird and highly implausible/impossible duplicate column/s, so we'll make a note of that here and deal with it later**

In [6]:
exp_df[[col for col in exp_df.columns if 'pal__lyx_ale' in col]].head()

,pal__lyx_ale2_f6__1,pal__lyx_ale2__1,pal__lyx_ale2__2,pal__lyx_ale4__1,pal__lyx_ale4__2
log-TPM,,,,,
b0002,9.627287,10.130315,10.155462,10.130315,10.155462
b0003,9.250534,9.708944,9.831138,9.708944,9.831138
b0004,9.203814,10.049444,10.190627,10.049444,10.190627
b0005,4.807384,5.772047,5.933463,5.772047,5.933463
b0006,6.398236,6.435048,6.220552,6.435048,6.220552


# Averaging gene expression values across replicates to clean up/simplify the data

## First getting gene expression averages between replicates and creating a new `dataframe` to hold this information

Thus just exploits the fact that replicates are denoted by "__x" in the sample names so we identify these and average them when possible. 

In [7]:
unique_cols = set(exp_df.columns.str[:-3])
new_exp_df = pd.DataFrame()

for i in unique_cols:
    new_exp_df[i] = exp_df[[col for col in exp_df.columns if col[:-3]==i]].mean(axis=1)
print(new_exp_df.shape)
new_exp_df.head()

(3923, 105)


/tmp/ipykernel_23143/3440901807.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_exp_df[i] = exp_df[[col for col in exp_df.columns if col[:-3]==i]].mean(axis=1)


,pal__arab_ale14,ica__thm_gal,ssw__xyl_ale3,fur__delfur_fe2,ytf__wt_ph5,42c__42c_ale8,ytf__delyheO,cra_crp__delcra_glc,cra_crp__delcra_ac,pgi__pgi_ale1,...,rpoB__rpoBE546V_lb,ssw__wt_glyc,ica__gth,glu__glu_ale4,efeU__menFentCubiC_ale36,fps__fps_ptsI_ale1,42c__42c_ale2,omics__bw_ac,ytf__delyeiE,pgi__pgi_ale2
log-TPM,,,,,,,,,,,,,,,,,,,,,
b0002,10.504998,11.090470,9.865774,11.221916,11.069910,10.997682,9.401156,11.230252,8.487038,9.816061,...,8.163975,7.678018,8.319211,11.038749,11.701351,9.955873,11.216042,9.219835,10.874816,9.783981
b0003,10.025917,10.402663,9.473220,10.879548,10.408418,10.747250,8.933743,11.106092,8.546123,9.163024,...,8.127386,8.384440,7.711763,10.443008,10.912852,9.536160,10.222039,8.963280,9.861151,8.990193
b0004,10.296601,10.633509,9.494152,10.174903,10.535924,10.247703,9.164724,10.486687,7.748146,9.134662,...,8.065428,8.745754,7.557831,10.551490,11.128015,9.797032,10.257204,8.770805,9.682288,9.040825
b0005,5.452944,4.686068,5.862383,6.331911,5.676045,6.417454,4.215120,5.818731,3.399998,5.705243,...,3.848184,5.925175,4.999832,6.360466,4.947044,4.861505,5.687336,4.597347,4.786873,4.690402
b0006,6.514145,6.663905,6.849183,6.839814,7.108237,7.046514,6.748235,6.937212,6.647096,7.159757,...,6.963816,6.186970,6.235536,6.816227,7.359833,6.910678,6.727618,6.488187,7.009695,7.552121


**Double checking the work**

Just making sure somethings add up here by taking an example column/condition and looking at the replicate values

In [8]:
example_col = list(unique_cols)[1]

exp_df[[col for col in exp_df.columns if col[:-3]==example_col]].head()

,ica__thm_gal__1,ica__thm_gal__2
log-TPM,,
b0002,11.092682,11.088257
b0003,10.394027,10.411298
b0004,10.601919,10.665099
b0005,4.512912,4.859224
b0006,6.723811,6.604000


And their average

In [9]:
new_exp_df[[example_col]].head()

,ica__thm_gal
log-TPM,
b0002,11.090470
b0003,10.402663
b0004,10.633509
b0005,4.686068
b0006,6.663905


## Dealing with the weird duplicate column/s

Time to kill any completely identical columns, these must be bugs on the data end and even though I could in theory keep one, their growth rate value is unclear (since it differs)

In [10]:
###Get an all-to-all correlation matrix between gene expression values
temp_corr = new_exp_df.corr(method='spearman')

In [11]:
###And make this into a symmetric dataframe
temp_df = pd.DataFrame(
    np.where(np.equal(*np.indices(temp_corr.shape)), np.nan, temp_corr.values),
    temp_corr.index, temp_corr.columns
)
print(temp_df.shape)
temp_df.head()

(105, 105)


,pal__arab_ale14,ica__thm_gal,ssw__xyl_ale3,fur__delfur_fe2,ytf__wt_ph5,42c__42c_ale8,ytf__delyheO,cra_crp__delcra_glc,cra_crp__delcra_ac,pgi__pgi_ale1,...,rpoB__rpoBE546V_lb,ssw__wt_glyc,ica__gth,glu__glu_ale4,efeU__menFentCubiC_ale36,fps__fps_ptsI_ale1,42c__42c_ale2,omics__bw_ac,ytf__delyeiE,pgi__pgi_ale2
pal__arab_ale14,NaN,0.880622,0.919062,0.915954,0.916830,0.867574,0.848017,0.922362,0.900065,0.918987,...,0.846204,0.845934,0.805012,0.891187,0.909152,0.924867,0.901711,0.931625,0.902226,0.875269
ica__thm_gal,0.880622,NaN,0.814639,0.845009,0.882751,0.801688,0.824915,0.888358,0.884646,0.846474,...,0.849325,0.801547,0.768087,0.882620,0.872295,0.898860,0.883579,0.903745,0.870752,0.860254
ssw__xyl_ale3,0.919062,0.814639,NaN,0.899916,0.901277,0.900105,0.857151,0.908824,0.851008,0.895229,...,0.779749,0.844000,0.796825,0.891669,0.900448,0.870549,0.902682,0.876793,0.895947,0.838098
fur__delfur_fe2,0.915954,0.845009,0.899916,NaN,0.951830,0.877977,0.844202,0.958872,0.924542,0.933114,...,0.807755,0.794648,0.869040,0.889556,0.942488,0.904895,0.900265,0.913742,0.904875,0.913807
ytf__wt_ph5,0.916830,0.882751,0.901277,0.951830,NaN,0.874055,0.891568,0.972327,0.915127,0.906419,...,0.836713,0.815491,0.874591,0.885708,0.964902,0.925843,0.892907,0.905586,0.956894,0.907995


**Identify columns that contain a value of "1." since this indicates they have a perfect correlation**

In [12]:
temp_df[temp_df.values==1]

,pal__arab_ale14,ica__thm_gal,ssw__xyl_ale3,fur__delfur_fe2,ytf__wt_ph5,42c__42c_ale8,ytf__delyheO,cra_crp__delcra_glc,cra_crp__delcra_ac,pgi__pgi_ale1,...,rpoB__rpoBE546V_lb,ssw__wt_glyc,ica__gth,glu__glu_ale4,efeU__menFentCubiC_ale36,fps__fps_ptsI_ale1,42c__42c_ale2,omics__bw_ac,ytf__delyeiE,pgi__pgi_ale2
pal__lyx_ale4,0.93004,0.883647,0.888263,0.878752,0.869489,0.828383,0.8044,0.88225,0.877665,0.898932,...,0.836452,0.815217,0.766783,0.882527,0.868086,0.917759,0.888757,0.924265,0.854234,0.852095
pal__lyx_ale2,0.93004,0.883647,0.888263,0.878752,0.869489,0.828383,0.8044,0.88225,0.877665,0.898932,...,0.836452,0.815217,0.766783,0.882527,0.868086,0.917759,0.888757,0.924265,0.854234,0.852095


Get rid of them both since there is obviously an error here somewhere

In [16]:
new_exp_df.drop(['pal__lyx_ale2', 'pal__lyx_ale4'], axis=1, inplace=True)
print(new_exp_df.shape)

(3923, 103)


# Average the growth rates across these replicates in the metadata as well

In [17]:
###Assign a unique id that removes the replicate information
meta_df['Simple_sample_id'] = meta_df['Sample ID'].str[:-3]
print(meta_df.shape)
###Group according to this new id
group_cols = ['Simple_sample_id']
###For these numeric columns I'll take the mean
metric_cols_a = ['Temperature (C)', 'pH', 'Growth Rate (1/hr)', 'Alignment']
aggs_a = meta_df.groupby(group_cols)[metric_cols_a].mean()
###And for these I'll just grab the count
metric_cols_b = ['Replicate #', 'Biological Replicates']
aggs_b = meta_df.groupby(group_cols)['Replicate #'].count()

###Drop the columns from the original dataframe (we'll add them back in later)
meta_df.drop(metric_cols_a, axis=1, inplace=True)
meta_df.drop(metric_cols_b, axis=1, inplace=True)
###And duplicates
meta_df.drop_duplicates(subset=group_cols, keep='first', inplace=True)

###Now merge the main dataframe with the grouped ones
meta_df = meta_df.merge(right=aggs_a, right_index=True, left_on=group_cols, how='right')
print(meta_df.shape)
meta_df = meta_df.merge(right=aggs_b, right_index=True, left_on=group_cols, how='right')
print(meta_df.shape)
meta_df.head()

(173, 27)
(105, 25)
(105, 26)


,Sample ID,Study,Project ID,Condition ID,Strain Description,Strain,Base Media,Carbon Source (g/L),Nitrogen Source (g/L),Electron Acceptor,...,Sequencing Machine,Additional Details,DOI,GEO,Simple_sample_id,Temperature (C),pH,Growth Rate (1/hr),Alignment,Replicate #
132,42c__42c_ale1__1,42C Evolution,42c,42c_ale1,Escherichia coli 42C.1.124.1,MG1655,M9,glucose(4),NH4Cl(1),O2,...,MiSeq,42C A1 F124 I1,doi.org/10.1093/molbev/msu209,GSE132442,42c__42c_ale1,42.0,7.0,0.95,98.40,1
140,42c__42c_ale10__1,42C Evolution,42c,42c_ale10,Escherichia coli 42C.10.153.1,MG1655,M9,glucose(4),NH4Cl(1),O2,...,MiSeq,42C A10 F153 I1,doi.org/10.1093/molbev/msu209,GSE132442,42c__42c_ale10,42.0,7.0,0.98,96.91,1
133,42c__42c_ale2__1,42C Evolution,42c,42c_ale2,Escherichia coli 42C.2.163.1,MG1655,M9,glucose(4),NH4Cl(1),O2,...,MiSeq,42C A2 F163 I1,doi.org/10.1093/molbev/msu209,GSE132442,42c__42c_ale2,42.0,7.0,0.97,97.51,1
134,42c__42c_ale3__1,42C Evolution,42c,42c_ale3,Escherichia coli 42C.3.120.1,MG1655,M9,glucose(4),NH4Cl(1),O2,...,MiSeq,42C A3 F120 I1,doi.org/10.1093/molbev/msu209,GSE132442,42c__42c_ale3,42.0,7.0,0.92,97.13,1
135,42c__42c_ale4__1,42C Evolution,42c,42c_ale4,Escherichia coli 42C.4.161.1,MG1655,M9,glucose(4),NH4Cl(1),O2,...,MiSeq,42C A4 F161 I1,doi.org/10.1093/molbev/msu209,GSE132442,42c__42c_ale4,42.0,7.0,1.03,97.07,1


**And get rid of those problematic samples from this dataframe as well**

In [18]:
print(meta_df.shape)
meta_df = meta_df[meta_df['Sample ID'].str.contains('pal__lyx_ale2__')==False]
print(meta_df.shape)
meta_df = meta_df[meta_df['Sample ID'].str.contains('pal__lyx_ale4__')==False]
print(meta_df.shape)

(105, 26)
(104, 26)
(103, 26)


**Sum should equal the original shape!**

In [19]:
meta_df['Replicate #'].value_counts()

2    61
1    41
6     1
Name: Replicate #, dtype: int64

**Make sure that the columns line up when matching across these two dataframes** 

In [20]:
new_exp_df = new_exp_df[meta_df['Simple_sample_id']]

In [21]:
assert all(new_exp_df.columns == meta_df['Simple_sample_id'])

**And add the doubling time just for good measure**

Which is just a slight transformation of growth rate

In [22]:
meta_df['Doubling_time'] = np.log(2)/meta_df['Growth Rate (1/hr)']

# Construct a third `dataframe` containing gene expression data summary stats

Strictly speaking this isn't super necessary but might as well do it now to get it done and over with

In [23]:
exp_summary_df = new_exp_df.apply(pd.DataFrame.describe, axis=1)

**The % signs seem to cause some problems down the road so lets remove them**

In [24]:
col_listy = []
for col in exp_summary_df.columns:
    if '%' not in col:
        col_listy.append(col)
    else:
        col_listy.append(col.replace('%', '_percentile'))
print(col_listy)
exp_summary_df.columns = col_listy

['count', 'mean', 'std', 'min', '25_percentile', '50_percentile', '75_percentile', 'max']


**And add some other dispersion metrics**

In [25]:
exp_summary_df['cv'] = exp_summary_df['std']/exp_summary_df['mean']
exp_summary_df['noise'] = exp_summary_df['std'].pow(2)/exp_summary_df['mean'].pow(2)

**Finally, adding some of (what we think are) the cool new variables to consider**

In [26]:
slopes = []
pearsons = []
spearmans = []
for gene in exp_summary_df.index:
    a, b, c, d, e = stats.linregress(new_exp_df.loc[gene], meta_df['Growth Rate (1/hr)'])
    slopes.append(a)
    pearsons.append(c)
    rho, p = stats.spearmanr(new_exp_df.loc[gene], meta_df['Growth Rate (1/hr)'])
    spearmans.append(rho)
    
exp_summary_df['lin_slope'] = slopes
exp_summary_df['lin_r'] = pearsons
exp_summary_df['spearmans_rho'] = spearmans

# Save some files

This was the whole point of all the code above. Should have some straightforward data tables now

In [25]:
print(new_exp_df.shape)
print(meta_df.shape)
print(exp_summary_df.shape)

(3923, 103)
(103, 27)
(3923, 13)


In [ ]:
new_exp_df.to_csv('../Data/processed_data/processed_expression_ecoli.tsv', sep='\t')
meta_df.to_csv('../Data/processed_data/processed_metadata_ecoli.tsv', sep='\t')
exp_summary_df.to_csv('../Data/processed_data/processed_expression_summary_ecoli.tsv', sep='\t')

# Get a thinned down dataset as a robustness check

Since conditions are correlated with one another, I'm constructing a more sparsely populated dataset where inter-condition correlations are minimized using a greedy algorithm. 

The algorithm works as follows:
1. Find which two conditions are the most highly correlated across the entire all-to-all correlation matrix
2. Randomly delete one of the two conditions in question
3. Iterate to (1)
4. Stop after reaching a pre-defined final dataset size

In [27]:
import random
random.seed(42)

In [28]:
cond_corr_mat = new_exp_df.corr(method='spearman')
cond_corr_mat.head()

,42c__42c_ale1,42c__42c_ale10,42c__42c_ale2,42c__42c_ale3,42c__42c_ale4,42c__42c_ale5,42c__42c_ale6,42c__42c_ale8,42c__42c_ale9,42c__wt_42c,...,ytf__delydcI_ph5,ytf__delydcI_ph8,ytf__delyddM,ytf__delyeiE,ytf__delyheO,ytf__delyiaJ,ytf__delyieP,ytf__wt_glc,ytf__wt_ph5,ytf__wt_ph8
42c__42c_ale1,1.000000,0.927120,0.882641,0.862337,0.890492,0.844707,0.934393,0.910083,0.853765,0.836816,...,0.806675,0.798028,0.828742,0.840311,0.902051,0.864096,0.889329,0.841736,0.803002,0.802468
42c__42c_ale10,0.927120,1.000000,0.931647,0.909807,0.963706,0.902406,0.919654,0.961923,0.890414,0.843208,...,0.836582,0.841167,0.846182,0.830256,0.861610,0.841320,0.898360,0.876208,0.832988,0.839818
42c__42c_ale2,0.882641,0.931647,1.000000,0.964109,0.956908,0.968556,0.856027,0.918062,0.963966,0.902696,...,0.899551,0.901670,0.888363,0.876668,0.827253,0.818446,0.865943,0.904547,0.892907,0.902017
42c__42c_ale3,0.862337,0.909807,0.964109,1.000000,0.938152,0.974434,0.858418,0.896590,0.965164,0.900097,...,0.897568,0.896120,0.888025,0.880415,0.811377,0.803547,0.851940,0.880488,0.888256,0.896066
42c__42c_ale4,0.890492,0.963706,0.956908,0.938152,1.000000,0.930396,0.887814,0.955125,0.919252,0.875380,...,0.869244,0.874356,0.863983,0.849296,0.835690,0.823237,0.879365,0.894170,0.863277,0.872864


**The `final_data_size` is of course completely arbitrary. But the point is to get rid of some correlated data so it does the trick.**

In [29]:
final_data_size = 30
n_to_prune = cond_corr_mat.shape[1] - final_data_size

In [30]:
np.fill_diagonal(cond_corr_mat.values, np.nan)
to_prune = []
for i in range(n_to_prune):
    tempy = cond_corr_mat.loc[[i for i in cond_corr_mat.columns if i not in to_prune]][[i for i in cond_corr_mat.columns if i not in to_prune]].max()
    to_prune.append(random.choice([tempy.sort_values(ascending=False).index[0],\
                                   tempy.sort_values(ascending=False).index[1]]))
np.fill_diagonal(cond_corr_mat.values, 1.)

**Remove relevant rows from two of the dataframes**

In [31]:
to_keep = [i for i in cond_corr_mat.columns if i not in to_prune]
new_exp_sprs_df = new_exp_df[to_keep]
meta_sprs_df = meta_df[meta_df['Simple_sample_id'].isin(to_keep)]

**And regenerate the summary dataframe to only use this thinned-down set of samples**

In [32]:
exp_summary_sprs_df = new_exp_sprs_df.apply(pd.DataFrame.describe, axis=1)
col_listy = []
for col in exp_summary_sprs_df.columns:
    if '%' not in col:
        col_listy.append(col)
    else:
        col_listy.append(col.replace('%', '_percentile'))
print(col_listy)
exp_summary_sprs_df.columns = col_listy

exp_summary_sprs_df['cv'] = exp_summary_sprs_df['std']/exp_summary_sprs_df['mean']
exp_summary_sprs_df['noise'] = exp_summary_sprs_df['std'].pow(2)/exp_summary_sprs_df['mean'].pow(2)

slopes = []
pearsons = []
spearmans = []
for gene in exp_summary_sprs_df.index:
    a, b, c, d, e = stats.linregress(new_exp_sprs_df.loc[gene], meta_sprs_df['Growth Rate (1/hr)'])
    slopes.append(a)
    pearsons.append(c)
    rho, p = stats.spearmanr(new_exp_sprs_df.loc[gene], meta_sprs_df['Growth Rate (1/hr)'])
    spearmans.append(rho)
    
exp_summary_sprs_df['lin_slope'] = slopes
exp_summary_sprs_df['lin_r'] = pearsons
exp_summary_sprs_df['spearmans_rho'] = spearmans

['count', 'mean', 'std', 'min', '25_percentile', '50_percentile', '75_percentile', 'max']


In [33]:
print(new_exp_sprs_df.shape)
print(meta_sprs_df.shape)
print(exp_summary_sprs_df.shape)

(3923, 30)
(30, 27)
(3923, 13)


In [ ]:
new_exp_sprs_df.to_csv('../Data/processed_data/processed_expression_ecoli_SPARSE.tsv', sep='\t')
meta_sprs_df.to_csv('../Data/processed_data/processed_metadata_ecoli_SPARSE.tsv', sep='\t')
exp_summary_sprs_df.to_csv('../Data/processed_data/processed_expression_summary_ecoli_SPARSE.tsv', sep='\t')

## Create neutral data sets for additional robustness check
**Limit analysis to non-ALE (adaptive laboratory environment) strains to control for selection**
**First, remove ALE strains in data sets**

In [34]:
meta_ntrl_df = meta_df[meta_df['Evolved Sample'].str.contains('No')==True]
print(meta_ntrl_df.shape)

(48, 27)


In [35]:
new_exp_ntrl_df = new_exp_df[meta_ntrl_df['Simple_sample_id']]
print(new_exp_ntrl_df.shape)

(3923, 48)


**And regenerate the summary dataframe to only use this non-ALE set of samples**

In [36]:
exp_summary_ntrl_df = new_exp_ntrl_df.apply(pd.DataFrame.describe, axis=1)
col_listy = []
for col in exp_summary_ntrl_df.columns:
    if '%' not in col:
        col_listy.append(col)
    else:
        col_listy.append(col.replace('%', '_percentile'))
print(col_listy)
exp_summary_ntrl_df.columns = col_listy

exp_summary_ntrl_df['cv'] = exp_summary_ntrl_df['std']/exp_summary_ntrl_df['mean']
exp_summary_ntrl_df['noise'] = exp_summary_ntrl_df['std'].pow(2)/exp_summary_ntrl_df['mean'].pow(2)

slopes = []
pearsons = []
spearmans = []
for gene in exp_summary_ntrl_df.index:
    a, b, c, d, e = stats.linregress(new_exp_ntrl_df.loc[gene], meta_ntrl_df['Growth Rate (1/hr)'])
    slopes.append(a)
    pearsons.append(c)
    rho, p = stats.spearmanr(new_exp_ntrl_df.loc[gene], meta_ntrl_df['Growth Rate (1/hr)'])
    spearmans.append(rho)
    
exp_summary_ntrl_df['lin_slope'] = slopes
exp_summary_ntrl_df['lin_r'] = pearsons
exp_summary_ntrl_df['spearmans_rho'] = spearmans

['count', 'mean', 'std', 'min', '25_percentile', '50_percentile', '75_percentile', 'max']


In [37]:
print(new_exp_ntrl_df.shape)
print(meta_ntrl_df.shape)
print(exp_summary_ntrl_df.shape)

(3923, 48)
(48, 27)
(3923, 13)


**Save the files for use later in the pipeline**

In [49]:
new_exp_ntrl_df.to_csv('../Data/processed_data/processed_expression_ecoli_NEUTRAL.tsv', sep='\t')
meta_ntrl_df.to_csv('../Data/processed_data/processed_metadata_ecoli_NEUTRAL.tsv', sep='\t')
exp_summary_ntrl_df.to_csv('../Data/processed_data/processed_expression_summary_ecoli_NEUTRAL.tsv', sep='\t')

**Limit analysis to non-ALE, non-mutants, and non-knock-outs**

**First, filter out mutants and knock-outs from neutral data sets**

In [40]:
# remove knock-outs
temp_meta_ntrl_wt_df = meta_ntrl_df[meta_ntrl_df['Strain Description'].str.contains('del')==False]
print(temp_meta_ntrl_wt_df.shape)

(32, 27)


In [43]:
# remove mutants
non_mut = ['Escherichia coli K-12 MG1655', 'Escherichia coli BW25113']
meta_ntrl_wt_df = temp_meta_ntrl_wt_df[temp_meta_ntrl_wt_df['Strain Description'].isin(non_mut)]
print(meta_ntrl_wt_df.shape)

(28, 27)


In [45]:
# remove knock-outs and mutants
new_exp_ntrl_wt_df = new_exp_df[meta_ntrl_wt_df['Simple_sample_id']]
print(new_exp_ntrl_wt_df.shape)

(3923, 28)


**And regenerate summary stats for WT neutral data set**

In [47]:
exp_summary_ntrl_wt_df = new_exp_ntrl_wt_df.apply(pd.DataFrame.describe, axis=1)
col_listy = []
for col in exp_summary_ntrl_wt_df.columns:
    if '%' not in col:
        col_listy.append(col)
    else:
        col_listy.append(col.replace('%', '_percentile'))
print(col_listy)
exp_summary_ntrl_wt_df.columns = col_listy

exp_summary_ntrl_wt_df['cv'] = exp_summary_ntrl_wt_df['std']/exp_summary_ntrl_wt_df['mean']
exp_summary_ntrl_wt_df['noise'] = exp_summary_ntrl_wt_df['std'].pow(2)/exp_summary_ntrl_wt_df['mean'].pow(2)

slopes = []
pearsons = []
spearmans = []
for gene in exp_summary_ntrl_wt_df.index:
    a, b, c, d, e = stats.linregress(new_exp_ntrl_wt_df.loc[gene], meta_ntrl_wt_df['Growth Rate (1/hr)'])
    slopes.append(a)
    pearsons.append(c)
    rho, p = stats.spearmanr(new_exp_ntrl_wt_df.loc[gene], meta_ntrl_wt_df['Growth Rate (1/hr)'])
    spearmans.append(rho)
    
exp_summary_ntrl_wt_df['lin_slope'] = slopes
exp_summary_ntrl_wt_df['lin_r'] = pearsons
exp_summary_ntrl_wt_df['spearmans_rho'] = spearmans

['count', 'mean', 'std', 'min', '25_percentile', '50_percentile', '75_percentile', 'max']


In [48]:
print(new_exp_ntrl_wt_df.shape)
print(meta_ntrl_wt_df.shape)
print(exp_summary_ntrl_wt_df.shape)

(3923, 28)
(28, 27)
(3923, 13)


**Save files for use later in pipeline**

In [50]:
new_exp_ntrl_wt_df.to_csv('../Data/processed_data/processed_expression_ecoli_NEUTRAL_WT.tsv', sep='\t')
meta_ntrl_wt_df.to_csv('../Data/processed_data/processed_metadata_ecoli_NEUTRAL_WT.tsv', sep='\t')
exp_summary_ntrl_wt_df.to_csv('../Data/processed_data/processed_expression_summary_ecoli_NEUTRAL_WT.tsv', sep='\t')